In [1]:
import logging
from noisepy.seis.io.compositerawstore import CompositeRawStore
from noisepy.seis.io.scedc_s3store import SCEDCS3DataStore, NCEDCS3DataStore
from noisepy.seis.io.pnwstore import PNWDataStore
from noisepy.seis.io.channelcatalog import XMLStationChannelCatalog
from datetimerange import DateTimeRange
from datetime import datetime, timezone  
from noisepy.seis.io.channel_filter_store import channel_filter

logger = logging.getLogger("noisepy.seis.io")
logger.setLevel("DEBUG")
logger.addHandler(logging.StreamHandler())

/home/niyiyu/anaconda3/envs/noisepy-io/lib/python3.10/site-packages/noisepy/seis/io/utils.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
SCEDC_DATA = "s3://scedc-pds/continuous_waveforms/"
NCEDC_DATA = "s3://ncedc-pds/continuous_waveforms/NC/"
PNW_DATA = "/1-fnp/pnwstore1/p-wd08/PNW2012/__/" 
PNW_DB_PATH="/fd1/yiyu_data/PNWstore_sqlite/2012.sqlite"

S3_STORAGE_OPTIONS = {"s3": {"anon": True}}
SCEDC_STATION_XML = "s3://scedc-pds/FDSNstationXML/CI/"  
NCEDC_STATION_XML = "s3://ncedc-pds/FDSNstationXML/NC/"
PNW_STATION_XML = "/1-fnp/pnwstore1/p-wd11/PNWStationXML/"

# timeframe for analysis
start = datetime(2012, 1, 1, tzinfo=timezone.utc)
end = datetime(2012, 1, 3, tzinfo=timezone.utc)
timerange = DateTimeRange(start, end)

In [3]:
scedc_stations = "RPV,SVD,BBR".split(",")  # SCEDC station
ncedc_stations = "KCT,KRP,KHMB".split(",") # NCEDC station
pnw_stations = "RATT,IRON,DBO".split(",") # UW station


scedc_catalog = XMLStationChannelCatalog(SCEDC_STATION_XML, 
                                         storage_options=S3_STORAGE_OPTIONS)
ncedc_catalog = XMLStationChannelCatalog(NCEDC_STATION_XML, "{network}.{name}.xml", 
                                         storage_options=S3_STORAGE_OPTIONS)
pnw_catalog = XMLStationChannelCatalog(PNW_STATION_XML, "{network}/{network}.{name}.xml")


scedc_store = SCEDCS3DataStore(SCEDC_DATA, scedc_catalog,  channel_filter(["CI"], scedc_stations, ["BHE", "BHN", "BHZ"]), 
                               timerange, storage_options=S3_STORAGE_OPTIONS)
ncedc_store = NCEDCS3DataStore(NCEDC_DATA, ncedc_catalog, channel_filter(["NC"], ncedc_stations, ["HHE", "HHN", "HHZ"]), 
                               timerange, storage_options=S3_STORAGE_OPTIONS)
pnw_store = PNWDataStore(PNW_DATA, pnw_catalog, PNW_DB_PATH, channel_filter(["UW", "UO"], pnw_stations, ["BHE", "BHN", "BHZ"]), 
                               timerange)

raw_store = CompositeRawStore({"CI": scedc_store, 
                               "NC": ncedc_store,
                               "UW": pnw_store,
                               "UO": pnw_store}) # Composite store for reading data

In [6]:
ts = raw_store.get_timespans()
print(ts)
channels = raw_store.get_channels(ts[0])
print(channels)
data = raw_store.read_data(timerange, channels[0])
print(data.data.shape)

[2012-01-01T00:00:00+0000 - 2012-01-02T00:00:00+0000, 2012-01-02T00:00:00+0000 - 2012-01-03T00:00:00+0000, 2012-01-01T00:00:00+0000 - 2012-01-02T00:00:00+0000, 2012-01-02T00:00:00+0000 - 2012-01-03T00:00:00+0000, 2012-01-01T00:00:00+0000 - 2012-01-02T00:00:00+0000, 2012-01-02T00:00:00+0000 - 2012-01-03T00:00:00+0000, 2012-01-01T00:00:00+0000 - 2012-01-02T00:00:00+0000, 2012-01-02T00:00:00+0000 - 2012-01-03T00:00:00+0000]


TIMING: 0.3577 secs. for Loading 1743 files from s3://scedc-pds/continuous_waveforms/2012/2012_001/
TIMING: 0.0243 secs. for Init: 1 timespans and 6 channels
Reading StationXML file s3://scedc-pds/FDSNstationXML/CI/CI_SVD.xml
Reading StationXML file s3://scedc-pds/FDSNstationXML/CI/CI_BBR.xml
Getting 6 channels for 2012-01-01T00:00:00+0000 - 2012-01-02T00:00:00+0000: [CI.BBR.BHE, CI.BBR.BHN, CI.BBR.BHZ, CI.SVD.BHE, CI.SVD.BHN, CI.SVD.BHZ]
TIMING: 0.4415 secs. for Loading 804 files from s3://ncedc-pds/continuous_waveforms/NC/2012/2012.001/
TIMING: 0.0118 secs. for Init: 1 timespans and 9 channels
Reading StationXML file s3://ncedc-pds/FDSNstationXML/NC/NC.KRP.xml
Reading StationXML file s3://ncedc-pds/FDSNstationXML/NC/NC.KCT.xml
Reading StationXML file s3://ncedc-pds/FDSNstationXML/NC/NC.KHMB.xml
Getting 9 channels for 2012-01-01T00:00:00+0000 - 2012-01-02T00:00:00+0000: [NC.KCT.HHE, NC.KCT.HHN, NC.KCT.HHZ, NC.KHMB.HHE, NC.KHMB.HHN, NC.KHMB.HHZ, NC.KRP.HHE, NC.KRP.HHN, NC.KRP.HHZ]
Read

[CI.BBR.BHE, CI.BBR.BHN, CI.BBR.BHZ, CI.SVD.BHE, CI.SVD.BHN, CI.SVD.BHZ, NC.KCT.HHE, NC.KCT.HHN, NC.KCT.HHZ, NC.KHMB.HHE, NC.KHMB.HHN, NC.KHMB.HHZ, NC.KRP.HHE, NC.KRP.HHN, NC.KRP.HHZ, UO.DBO.BHE, UO.DBO.BHN, UO.DBO.BHZ, UW.IRON.BHE, UW.IRON.BHN, UW.IRON.BHZ, UW.RATT.BHE, UW.RATT.BHN, UW.RATT.BHZ, UO.DBO.BHE, UO.DBO.BHN, UO.DBO.BHZ, UW.IRON.BHE, UW.IRON.BHN, UW.IRON.BHZ, UW.RATT.BHE, UW.RATT.BHN, UW.RATT.BHZ]


TIMING: 0.2515 secs. for Loading 1743 files from s3://scedc-pds/continuous_waveforms/2012/2012_002/
TIMING: 0.0240 secs. for Init: 2 timespans and 18 channels


(108,)
